In [ ]:
import numpy as np
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC
from pprint import pprint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Support-Vector-Machines-from-Scratch/data/training.csv")
train_data_df = pd.read_csv("/content/drive/My Drive/Support-Vector-Machines-from-Scratch/data/training.csv") 
test_data_df = pd.read_csv("/content/drive/My Drive/Support-Vector-Machines-from-Scratch/data/testing.csv")

train_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  623 non-null    int64  
 1   Survived     623 non-null    int64  
 2   Pclass       623 non-null    int64  
 3   Name         623 non-null    object 
 4   Sex          623 non-null    object 
 5   Age          492 non-null    float64
 6   SibSp        623 non-null    int64  
 7   Parch        623 non-null    int64  
 8   Ticket       623 non-null    object 
 9   Fare         623 non-null    float64
 10  Cabin        140 non-null    object 
 11  Embarked     622 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 58.5+ KB


In [ ]:
# Age
MedianAge = train_data_df.Age.median()
train_data_df.Age = train_data_df.Age.fillna(value=MedianAge)
# Embarked replace NaN with the mode value
ModeEmbarked = train_data_df.Embarked.mode()[0]
train_data_df.Embarked = train_data_df.Embarked.fillna(value=ModeEmbarked)
# Fare have 1 NaN missing value on the Submission dataset
MedianFare = train_data_df.Fare.median()
train_data_df.Fare = train_data_df.Fare.fillna(value=MedianFare)

train_data_df['Sex'] = train_data_df['Sex'].map({'male': 0, 'female': 1}).astype(int)
test_data_df['Sex'] = test_data_df['Sex'].map({'male': 0, 'female': 1}).astype(int)

train_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  623 non-null    int64  
 1   Survived     623 non-null    int64  
 2   Pclass       623 non-null    int64  
 3   Name         623 non-null    object 
 4   Sex          623 non-null    int64  
 5   Age          623 non-null    float64
 6   SibSp        623 non-null    int64  
 7   Parch        623 non-null    int64  
 8   Ticket       623 non-null    object 
 9   Fare         623 non-null    float64
 10  Cabin        140 non-null    object 
 11  Embarked     623 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 58.5+ KB


In [ ]:
title = ['Mlle','Mrs', 'Mr', 'Miss','Master','Don','Rev','Dr','Mme','Ms','Major','Col','Capt','Countess']

def ExtractTitle(name):
    tit = 'missing'
    for item in title :
        if item in name:
          if item == title[9]:
            tit =  title[3]
          elif item == title[11] or item == title[13]:
            tit =  title[10]
          elif item == title[0] or item == title[8]:
            tit =  title[3]
          else:  
            tit = item
    if tit == 'missing':
        tit = 'Mr'
    return tit

train_data_df["Title"] = train_data_df.apply(lambda row: ExtractTitle(row["Name"]),axis=1)

# Cabin
train_data_df["Cabin"] = train_data_df.apply(lambda obs: "No" if pd.isnull(obs['Cabin']) else "Yes", axis=1)
train_data_df = pd.get_dummies(train_data_df,drop_first=True,columns=['Sex','Title','Cabin','Embarked'])

scale = StandardScaler().fit(train_data_df[['Age', 'Fare']])
train_data_df[['Age', 'Fare']] = scale.transform(train_data_df[['Age', 'Fare']])
Target = train_data_df.Survived
Features = train_data_df.drop(['Survived','Name','Ticket','PassengerId'],axis=1)

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(Features, Target, test_size = 0.3, random_state=42)
Target = train_data_df.Survived

print(X_train.shape)

svm = SVC(probability=True)
# Fit to the training data
svm.fit(X_train, y_train)
# Predict on the test data: y_pred
y_pred = svm.predict(X_test)

# train_data_df = train_data_df[train_data_df.Age.notnull()][['Age','Sex',  'Pclass', 'Parch','Fare', 'SibSp','Survived','Embarked','Cabin']]

X_train
# LE = LabelEncoder()

# train_data_df['Cabin'] = LE.fit_transform(train_data_df['Cabin'])
# test_data_df['Cabin'] = LE.fit_transform(test_data_df['Cabin'])
# train_data_df['Embarked'] = LE.fit_transform(train_data_df['Embarked'])
# test_data_df['Embarked'] = LE.fit_transform(test_data_df['Embarked'])

(436, 15)
Score Support Vector Machines on train data : 0.8302752293577982


,Pclass,Age,SibSp,Parch,Fare,Sex_1,Title_Dr,Title_Major,Title_Master,Title_Miss,Title_Mr,Title_Rev,Cabin_Yes,Embarked_Q,Embarked_S
412,1,0.281090,1,0,1.272981,1,0,0,0,1,0,0,1,1,0
437,2,-0.422828,2,3,-0.283482,1,0,0,0,0,1,0,0,0,1
33,2,2.862121,0,0,-0.463705,0,0,0,0,0,1,0,0,0,1
0,3,-0.579254,1,0,-0.534701,0,0,0,0,0,1,0,0,0,1
615,2,-0.422828,1,2,0.726854,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,3,-1.048532,5,2,0.331457,1,0,0,0,1,0,0,0,0,1
106,3,-0.657467,0,0,-0.525963,1,0,0,0,1,0,0,0,0,1
270,1,-0.109975,0,0,-0.015880,0,0,0,0,0,1,0,0,0,1
435,1,-1.204958,1,2,1.928334,1,0,0,0,1,0,0,1,0,1


In [ ]:
test_data_df["Title"] = test_data_df.apply(lambda row: ExtractTitle(row["Name"]),axis=1)

# Cabin
test_data_df["Cabin"] = test_data_df.apply(lambda obs: "No" if pd.isnull(obs['Cabin']) else "Yes", axis=1)
test_data_df = pd.get_dummies(test_data_df,drop_first=True,columns=['Sex','Title','Cabin','Embarked'])

scale = StandardScaler().fit(test_data_df[['Age', 'Fare']])
test_data_df[['Age', 'Fare']] = scale.transform(test_data_df[['Age', 'Fare']])

test_data_df['Age'] = test_data_df['Age'].fillna(test_data_df['Age'].median())
test_data = test_data_df.drop(['Name','Ticket','PassengerId'],axis=1)

# Create training and test set

y_pred = svm.predict(test_data)
test_data

,Pclass,Age,SibSp,Parch,Fare,Sex_1,Title_Dr,Title_Major,Title_Master,Title_Miss,Title_Mr,Title_Rev,Cabin_Yes,Embarked_Q,Embarked_S
0,3,-0.573799,0,0,-0.441346,0,0,0,0,0,1,0,0,0,1
1,3,-0.573799,0,0,-0.298403,0,0,0,0,0,1,0,0,0,1
2,1,2.120477,0,0,-0.017212,0,0,0,0,0,1,0,1,0,1
3,2,1.851050,0,0,-0.363410,0,0,0,0,0,0,1,0,1,0
4,1,-0.573799,0,0,0.773924,1,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,2,-0.169657,0,0,-0.352142,0,0,0,0,0,0,1,0,0,1
264,1,-0.708513,0,0,-0.057444,1,0,0,0,1,0,0,1,0,1
265,3,-0.102301,1,2,-0.170989,1,0,0,0,1,0,0,0,0,1
266,1,-0.237014,0,0,-0.057444,0,0,0,0,0,1,0,1,0,0


In [ ]:
arr = np.array(y_pred)
output = pd.DataFrame({'id': test_data_df.PassengerId, 'solution': y_pred})

print(output)
output.to_csv('/content/drive/My Drive/Support-Vector-Machines-from-Scratch/data/sample.csv', index=False)

      id  solution
0      1         0
1      2         0
2      3         0
3      4         0
4      5         1
..   ...       ...
263  264         0
264  265         1
265  266         1
266  267         0
267  268         0

[268 rows x 2 columns]
